In [1]:
import itertools

import numpy as np
import pandas as pd

## Helpers

In [2]:
def string_replace_ruleset(ruleset: dict[str, str] = {}):
    def inner_func(s) -> str:
        if s is None or s is np.nan or s == "" or s == "-":
            return np.nan

        for replace_with, find_what in ruleset.items():
            for substring in find_what:
                if substring.lower() in s.lower():
                    return replace_with

        return s.strip()

    return inner_func

## SERP Dataset

In [3]:
df_serp = pd.read_json("../data/scrape-browser/lkpp-serp.jsonl", lines=True)
df_serp.head()

,name,tags,details,page,category,commodity,product_url,image_url,scrape_url
0,Laptop Administrasi Perkantoran Axioo,"[Produk Dalam Negeri, Verified Vendor, UMKK, N...","{'provider': 'CAESAR SAPTA PRATAMA', 'tkdn_per...",1,1237626,90424,https://e-katalog.lkpp.go.id/katalog/produk/de...,https://e-katalog.lkpp.go.id/katalog/produk/do...,https://e-katalog.lkpp.go.id/productsearchcont...
1,ACER Laptop Travelmate P214 Core i5/8 GB/512 G...,"[Produk Dalam Negeri, Verified Vendor, UMKK, N...","{'provider': 'Cemerlang Air Cond', 'tkdn_perce...",1,1237626,90424,https://e-katalog.lkpp.go.id/katalog/produk/de...,https://e-katalog.lkpp.go.id/katalog/produk/do...,https://e-katalog.lkpp.go.id/productsearchcont...
2,ACER Laptop Travelmate P214 Core i3/8 GB/512 G...,"[Produk Dalam Negeri, Verified Vendor, UMKK, N...","{'provider': 'Cemerlang Air Cond', 'tkdn_perce...",1,1237626,90424,https://e-katalog.lkpp.go.id/katalog/produk/de...,https://e-katalog.lkpp.go.id/katalog/produk/do...,https://e-katalog.lkpp.go.id/productsearchcont...
3,AXIOO Laptop Mybook Pro Core i5 L5 (8N5),"[Produk Dalam Negeri, Verified Vendor, UMKK, N...","{'provider': 'Cemerlang Air Cond', 'tkdn_perce...",1,1237626,90424,https://e-katalog.lkpp.go.id/katalog/produk/de...,https://e-katalog.lkpp.go.id/katalog/produk/do...,https://e-katalog.lkpp.go.id/productsearchcont...
4,AXIOO LAPTOP MYBOOK 14F,"[Produk Dalam Negeri, Verified Vendor, UMKK, N...","{'provider': 'PT STATION SUMBER MAKMUR JAYA', ...",1,1237626,90424,https://e-katalog.lkpp.go.id/katalog/produk/de...,https://e-katalog.lkpp.go.id/katalog/produk/do...,https://e-katalog.lkpp.go.id/productsearchcont...


In [4]:
# derive ID from URL
df_serp["id"] = df_serp["product_url"].str.extract(r"\/(\d+)\?")

In [5]:
# one-hot encode tags
product_tags = list(set(itertools.chain(*df_serp["tags"].tolist())) - set([None]))
product_tags_cols = ["tag_" + x.lower().replace(" ", "_") for x in product_tags]

df_serp[product_tags_cols] = df_serp.apply(lambda x: [True if v in x["tags"] else False for v in product_tags], axis=1, result_type='expand')

In [6]:
# drop unnecessary columns
df_product_tags = df_serp[["id", *product_tags_cols]]
df_product_tags.head()

,id,tag_umkk,tag_verified_vendor,tag_official_vendor,tag_produk_dalam_negeri
0,83451581,True,True,False,True
1,84521755,True,True,False,True
2,84521452,True,True,False,True
3,83971989,True,True,False,True
4,80375727,True,True,False,True


## Product Detail Dataset

In [7]:
df = pd.read_json("../data/scrape-browser/lkpp-products.jsonl", lines=True)
df.head()

,title,sku,price,stock,tags,details,url,alerts
0,LAPTOP ADMINISTRASI PERKANTORAN AXIOO,4522100001-LTN-224654966,"Rp 12.500.000,00",Stok Produk : 5,"[CAESAR SAPTA PRATAMA, Katalog Nasional, Peral...",{'Nama Produk': 'Laptop Administrasi Perkantor...,https://e-katalog.lkpp.go.id/katalog/produk/de...,Produk ini Tidak Aktif dan sedang dilaporkan m...
1,ACER LAPTOP TRAVELMATE P214 CORE I5/8 GB/512 G...,4522100001-LTN-230428659,"Rp 18.300.000,00",Stok Produk : 312,"[Cemerlang Air Cond, Katalog Nasional, Peralat...",{'Nama Produk': 'Laptop Travelmate P214 Core i...,https://e-katalog.lkpp.go.id/katalog/produk/de...,NaN
2,ACER LAPTOP TRAVELMATE P214 CORE I3/8 GB/512 G...,4522100001-LTN-230427140,"Rp 15.070.000,00",Stok Produk : 312,"[Cemerlang Air Cond, Katalog Nasional, Peralat...",{'Nama Produk': 'Laptop Travelmate P214 Core i...,https://e-katalog.lkpp.go.id/katalog/produk/de...,NaN
3,AXIOO LAPTOP MYBOOK PRO CORE I5 L5 (8N5),4522100001-LTN-227419165,"Rp 18.100.000,00",Stok Produk : 211,"[Cemerlang Air Cond, Katalog Nasional, Peralat...",{'Nama Produk': 'Laptop Mybook Pro Core i5 L5 ...,https://e-katalog.lkpp.go.id/katalog/produk/de...,Produk ini Tidak Aktif dan sedang dilaporkan m...
4,AXIOO LAPTOP MYBOOK 14F,4522100001-LTN-208199795,"Rp 3.999.000,00",Stok Produk : 1,"[PT STATION SUMBER MAKMUR JAYA, Katalog Nasion...","{'Nama Produk': 'LAPTOP MYBOOK 14F', 'Masa Ber...",https://e-katalog.lkpp.go.id/katalog/produk/de...,Produk ini Tidak Aktif dan sedang dilaporkan m...


In [8]:
df = df.join(pd.json_normalize(df["details"]))
df.head()

,title,sku,price,stock,tags,details,url,alerts,Nama Produk,Masa Berlaku Produk,...,RAM,Storage,VGA,Ukuran Layar,Koneksi,Deskripsi,Warna,Negara Asal Pembuatan Produk,Label Produk Ramah Lingkungan Hidup,Keterangan Lainnya
0,LAPTOP ADMINISTRASI PERKANTORAN AXIOO,4522100001-LTN-224654966,"Rp 12.500.000,00",Stok Produk : 5,"[CAESAR SAPTA PRATAMA, Katalog Nasional, Peral...",{'Nama Produk': 'Laptop Administrasi Perkantor...,https://e-katalog.lkpp.go.id/katalog/produk/de...,Produk ini Tidak Aktif dan sedang dilaporkan m...,Laptop Administrasi Perkantoran Axioo,2024-12-31,...,8 Gb,512 Gb,Intel Iris Plus Graphic,14 inci,Realtek 8821CE ac/b/g/n & Bluetooth 5.0,- Processor: Intel Core™ I5- 3.70 GHz - Frekue...,Grey,Indonesia,-,-
1,ACER LAPTOP TRAVELMATE P214 CORE I5/8 GB/512 G...,4522100001-LTN-230428659,"Rp 18.300.000,00",Stok Produk : 312,"[Cemerlang Air Cond, Katalog Nasional, Peralat...",{'Nama Produk': 'Laptop Travelmate P214 Core i...,https://e-katalog.lkpp.go.id/katalog/produk/de...,NaN,Laptop Travelmate P214 Core i5/8 GB/512 GB (TM...,2025-12-31,...,8 GB DDR4,512GB SSD,Integrated Graphics,14 inch,"LAN, WiFi, Bluetooth",Travelmate P214 Core-i5 8G 512GB SSD 14 Inch W...,-,Indonesia,-,Garansi 2 tahun part & 3 tahun labor
2,ACER LAPTOP TRAVELMATE P214 CORE I3/8 GB/512 G...,4522100001-LTN-230427140,"Rp 15.070.000,00",Stok Produk : 312,"[Cemerlang Air Cond, Katalog Nasional, Peralat...",{'Nama Produk': 'Laptop Travelmate P214 Core i...,https://e-katalog.lkpp.go.id/katalog/produk/de...,NaN,Laptop Travelmate P214 Core i3/8 GB/512 GB (TM...,2025-12-31,...,8GB DDR4,512GB SSD,Intel Graphics,14 Inch,"LAN, WiFi, Bluetooth","Travelmate P214, Core-i3, 8G, 512GB SSD, 14 In...",-,Indonesia,-,Garansi 2 tahun part & 3 tahun labor
3,AXIOO LAPTOP MYBOOK PRO CORE I5 L5 (8N5),4522100001-LTN-227419165,"Rp 18.100.000,00",Stok Produk : 211,"[Cemerlang Air Cond, Katalog Nasional, Peralat...",{'Nama Produk': 'Laptop Mybook Pro Core i5 L5 ...,https://e-katalog.lkpp.go.id/katalog/produk/de...,Produk ini Tidak Aktif dan sedang dilaporkan m...,Laptop Mybook Pro Core i5 L5 (8N5),2025-12-31,...,8 GB NVME,512 GB NVME,Integrated,14 Inch,"Koneksi LAN, WiFi, Bluetooth","Notebook Axioo Mybook Pro L5 (8N5), Layar 14 i...",-,Indonesia,Produk Ramah Lingkungan Hidup,Garansi 1 tahun
4,AXIOO LAPTOP MYBOOK 14F,4522100001-LTN-208199795,"Rp 3.999.000,00",Stok Produk : 1,"[PT STATION SUMBER MAKMUR JAYA, Katalog Nasion...","{'Nama Produk': 'LAPTOP MYBOOK 14F', 'Masa Ber...",https://e-katalog.lkpp.go.id/katalog/produk/de...,Produk ini Tidak Aktif dan sedang dilaporkan m...,LAPTOP MYBOOK 14F,2025-12-31,...,6GB,256GB SSD,Integrated Intel UHD Graphics,13.3 INCH IPS 2.5K 2560X1600,"WiFi + BT4.0, HiRes Webcam",Spesifikasi : Processor : Intel Celeron N4020 ...,"HITAM, BIRU, MERAH",INDONESIA,-,-


In [9]:
df_norm = df.copy()

### General Data

In [10]:
# derive ID from URL
df_norm["id"] = df_norm["url"].str.extract(r"\/(\d+)\?")

In [11]:
# derive product name
# df_norm["product_name"] = df_norm["Nama Produk"].str.strip()

In [12]:
# derive stock
df_norm["stock_parsed"] = df_norm["stock"].str.extract(r"(\d+)").astype(float)

In [13]:
# derive price
df_norm["price_parsed"] = (
    df_norm["price"]
    .str.replace(".", "")
    .str.replace(",", ".")
    .str.replace("Rp ", "")
    .astype(float)
)

In [14]:
# derive expiry date
df_norm["expiry_date"] = pd.to_datetime(df_norm["Masa Berlaku Produk"], errors="coerce")

In [15]:
# derive scores
df_norm["bmp_score"] = df_norm["Nilai BMP"].replace("-", None).astype(float)
df_norm["tkdn_score"] = df_norm["Nilai TKDN(%)"].replace("Tidak Ada", None).astype(float)
df_norm["tkdn_bmp_score"] = df_norm["Nilai TKDN + Nilai BMP"].astype(float)

In [16]:
# derive unit measurement
df_norm["unit_measurement"] = df_norm["Unit Pengukuran"].str.lower().str.strip()

In [17]:
# derive codes
df_norm["kbki_code"] = df_norm["Kode KBKI"].str.strip()
df_norm["sni_number"] = df_norm["No. SNI"].str.strip().replace("Tidak Ada", None)

In [18]:
# derive warnings
df_norm["is_inactive"] = ~df_norm["alerts"].isna()
df_norm["warning_wrong_category"] = df_norm["alerts"].str.contains("Salah Kategori", na=False)
df_norm["warning_wrong_description"] = df_norm["alerts"].str.contains("Ketidaksesuaian Informasi Produk", na=False)

### Brand and Company

In [19]:
# derive brand
df_norm["brand_status"] = df_norm["Status Merek"].replace("-", None)

df_norm["brand_parsed"] = (
    df_norm["Merek"]
    .str.upper()
    .str.replace("(\+|LOGO|PONGO|MYPC|MYBOOK)", "", regex=True)
    .str.replace("LIBERAL", "LIBERA")
    .str.replace("MACBOOK", "APPLE")
    .str.strip()
    .replace("TANPA MEREK", None)
)

In [20]:
# derive company name
ruleset_company_names = {
    "TERA DATA": ["PT TERA DATA INDONUSA TBK"],
}

df_norm["certificate_holder_company"] = (
    df_norm["Nama Pemilik Merek"]
    .replace("-", None)
    .str.upper()
)

df_norm["brand_holder_company"] = (
    df_norm["Nama Pemilik Merek"]
    .replace("-", None)
    .str.upper()
    .str.replace("PT.", "PT")
    .str.replace("CV.", "CV")
    .apply(string_replace_ruleset(ruleset_company_names))
)

df_norm["provider_product_number"] = df_norm["No. Produk Penyedia"].str.strip()
df_norm["provider_company"] = (
    df_norm["tags"].apply(lambda x: x[0])
    .replace("-", None)
    .str.upper()
    .str.replace("PT.", "PT")
    .str.replace("CV.", "CV")
    .apply(string_replace_ruleset(ruleset_company_names))
)

In [21]:
# derive manufacturer
ruleset_manufacturer_country = {
    "Indonesia": ["indo", "idn", "pdn", "jakarta", "lokal"],
    "Swiss": ["swiss"],
    "Korea": ["korea"],
    "Taiwan": ["taiwan"],
    "Tiongkok": ["tiongk"],
    "Cina": ["china", "cina"],
    "Jepang": ["jepang", "japan"],
    "Amerika Serikat": ["amerika", "california", "usa"],
    "Lainnya": ["ada", "import", "--", "n/a", "star black", "hitam", "tidak", "."],
}

df_norm["manufacturing_country"] = df_norm["Negara Asal Pembuatan Produk"].apply(
    string_replace_ruleset(ruleset_manufacturer_country)
)

### Laptop Specs

In [22]:
# memory
df_norm["memory"] = df_norm["RAM"].str.extract("(\d+) ?GB", expand=False).astype(float)

In [23]:
# storage
def derive_storage(x):
    if x is None or pd.isna(x[0]) or pd.isna(x[1]):
        return None
    
    if "T" in x[1]:
        return int(x[0]) * 1024
    
    return int(x[0])

df_norm["storage"] = df_norm["Storage"].str.extract("(\d+) ?(GB?|TB?)", expand=True).apply(derive_storage, axis=1)

In [24]:
# merge with tags
df_norm = df_norm.merge(df_product_tags, on="id")

# drop raw columns
df_norm = df_norm.drop(columns=[
    "price",
    "stock",
    "tags",
    "details",
    "alerts"
])

# rename columns
df_norm = df_norm.rename(columns={
    # title
    "title": "product_name",

    # original columns
    "Type/Model": "model",
    "Processor": "processor",
    # "RAM": "memory",
    # "Storage": "storage",
    "VGA": "graphics_card",
    "Ukuran Layar": "screen_size",
    "Koneksi": "connectivity",
    "Warna": "color",
    "Deskripsi": "description",
    "Keterangan Lainnya": "extra_description",

    # parsed columns
    "stock_parsed": "stock",
    "price_parsed": "price",
    "brand_parsed": "brand"
})

In [25]:
# select relevant columns
sel_cols = [
    # general data
    "id", "sku", "product_name", "price", "stock", "unit_measurement",

    # LKPP detail
    "is_inactive", "expiry_date", "bmp_score", "tkdn_score", "tkdn_bmp_score", "kbki_code", "sni_number", 
    "brand", "brand_status", "brand_holder_company", "certificate_holder_company", 
    "provider_company", "provider_product_number", "manufacturing_country",

    # laptop specs
    "model", "processor", "memory", "storage", "graphics_card", "screen_size", "connectivity", "color", 
    
    # extra data
    "warning_wrong_category", "warning_wrong_description", *product_tags_cols,
    "description", "extra_description", "url"
]

# copy and drop duplicates
df_final = df_norm[sel_cols].drop_duplicates(subset=["id"])
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1964 entries, 0 to 3271
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          1964 non-null   object        
 1   sku                         1964 non-null   object        
 2   product_name                1964 non-null   object        
 3   price                       1964 non-null   float64       
 4   stock                       1894 non-null   float64       
 5   unit_measurement            1964 non-null   object        
 6   is_inactive                 1964 non-null   bool          
 7   expiry_date                 1963 non-null   datetime64[ns]
 8   bmp_score                   1753 non-null   float64       
 9   tkdn_score                  1933 non-null   float64       
 10  tkdn_bmp_score              1933 non-null   float64       
 11  kbki_code                   1964 non-null   object        
 1

In [26]:
# save to file
df_final.to_json("../data/clean/lkpp-products.jsonl", orient="records", lines=True)
df_final.head()

,id,sku,product_name,price,stock,unit_measurement,is_inactive,expiry_date,bmp_score,tkdn_score,...,color,warning_wrong_category,warning_wrong_description,tag_umkk,tag_verified_vendor,tag_official_vendor,tag_produk_dalam_negeri,description,extra_description,url
0,83451581,4522100001-LTN-224654966,LAPTOP ADMINISTRASI PERKANTORAN AXIOO,12500000.0,5.0,unit,True,2024-12-31,NaN,99.64,...,Grey,True,True,True,True,False,True,- Processor: Intel Core™ I5- 3.70 GHz - Frekue...,-,https://e-katalog.lkpp.go.id/katalog/produk/de...
1,84521755,4522100001-LTN-230428659,ACER LAPTOP TRAVELMATE P214 CORE I5/8 GB/512 G...,18300000.0,312.0,unit,False,2025-12-31,NaN,99.04,...,-,False,False,True,True,False,True,Travelmate P214 Core-i5 8G 512GB SSD 14 Inch W...,Garansi 2 tahun part & 3 tahun labor,https://e-katalog.lkpp.go.id/katalog/produk/de...
2,84521452,4522100001-LTN-230427140,ACER LAPTOP TRAVELMATE P214 CORE I3/8 GB/512 G...,15070000.0,312.0,unit,False,2025-12-31,NaN,99.04,...,-,False,False,True,True,False,True,"Travelmate P214, Core-i3, 8G, 512GB SSD, 14 In...",Garansi 2 tahun part & 3 tahun labor,https://e-katalog.lkpp.go.id/katalog/produk/de...
3,83971989,4522100001-LTN-227419165,AXIOO LAPTOP MYBOOK PRO CORE I5 L5 (8N5),18100000.0,211.0,unit,True,2025-12-31,NaN,99.04,...,-,False,True,True,True,False,True,"Notebook Axioo Mybook Pro L5 (8N5), Layar 14 i...",Garansi 1 tahun,https://e-katalog.lkpp.go.id/katalog/produk/de...
4,80375727,4522100001-LTN-208199795,AXIOO LAPTOP MYBOOK 14F,3999000.0,1.0,unit,True,2025-12-31,NaN,99.04,...,"HITAM, BIRU, MERAH",False,True,True,True,False,True,Spesifikasi : Processor : Intel Celeron N4020 ...,-,https://e-katalog.lkpp.go.id/katalog/produk/de...
